# Pytorch ile Kendi Datasetinden Eğitim


Bir çok tutorialda torchvision dataset içinden hızlıca bir dataset çekmek tercih edilir çünkü çok kolayca düzgün bir dataset oluşturmuş olursun.

Biz bugün kendi elimizdeki bir data seti kullanarak bu işlemleri yapacağız.

Bunun için bir bulk downloader uzantısı ile googledan bir çok resmi hızlıca kayıt edebilir ya da elimizdeki hazır bir seti kullanabiliriz. Araştırmalar sırasında video çekip bu videodan bir python scripti ile ekran görüntüleri almak gibi teknikler ile de veri setleri hızlıca oluşturulabilir

ancak data bir miktar düzenlemeye ihtiyaç duyabilir.



# Database yapısı


Oldukça basit bir yapıyı takip edeceğiz 
                                      
data/                                           
├─ cat/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             
├─ rabbit/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             
├─ squirrel/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             

Bu dosya hem zipli hem de sıkıştırılmamış olarak repoda mevcut. Zipli hali colabda çalışırken wget ile kolayca indirmen için

Colabde isen kolayca indir

In [ ]:
!wget https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CatRabbit/data.zip

In [ ]:
!unzip -q data.zip

# Define Transforms

Transfromların olay resimlere rastgele değişimler uygulamaktır.  

Bu hem elimizdeki data miktarını arttırmak için güzel bir metot hem de farklı koşullar altında da tanıma yapabilmek için önemli.

Örneğin rastgele döndürmeler eklemek sayesinde eğer kedi baş aşağı durursa da onu tanıyacağız.





In [ ]:
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5)
])

# Data setimizi aktarmak

eğer data setimiz kurallara uygun bir yapıda ise işimiz kolay. Dosya yapısı aşağıdaki gibi ise süreç çok kolay.
              
data/                                           
├─ cat/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             
├─ rabbit/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             

In [ ]:
from torchvision.datasets import ImageFolder
train_dataset = torchvision.datasets.ImageFolder(root='data', transform=transform)

root data adressi  
transform ise önceden belirtilen transformlardır. 

In [ ]:
from torch.utils.data import Dataset
from torchvision.io import read_image, ImageReadMode

In [ ]:
import glob
import os


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.transform = transform
        self.image_paths = []
        # kullandığın tüm uzantıları formatları buraya ekle 
        for ext in ['png', 'jpg']:
            self.image_paths += glob.glob(os.path.join(root_dir, '*', f'*.{ext}'))
        class_set = set()
        for path in self.image_paths:
            class_set.add(os.path.dirname(path))
        self.class_lbl = { cls: i for i, cls in enumerate(sorted(list(class_set)))}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = read_image(self.image_paths[idx], ImageReadMode.RGB).float()
        cls = os.path.basename(os.path.dirname(self.image_paths[idx]))
        label = self.class_lbl[cls]

        return self.transform(img), torch.tensor(label)